In [1]:
import time
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

In [2]:
# Loading dataset
file = pd.read_csv("VNName_Final.csv", encoding='utf-8')
print(file.shape)

(1147651, 2)


In [3]:
file.head(2)

,full_name,gender
0,Bành Thị Bảo Châu,1
1,Biện Nguyễn Anh Thư,1


In [4]:
X = file['full_name']
Y = file['gender']


In [5]:
# show number of male and female
file['gender'].value_counts()

0    582997
1    564654
Name: gender, dtype: int64

In [6]:
X[1]

'Biện Nguyễn Anh Thư'

In [7]:
# Tiền xử lý dữ liệu cho full name
documents = []
words = 0

for sen in range(0, len(X)):
    
    # Dòng này đếm số lượng ký tự
    words += len(str(X[sen]).split())
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
        
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
       
    documents.append(document)

In [8]:
print(words)

4015734


In [9]:
# Using 1-grams and 2-grams 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

bigram_vectorizer = CountVectorizer(analyzer="word",ngram_range=(3, 3),token_pattern=r'\b\w+\b', min_df=1,max_features=300)

X = bigram_vectorizer.fit_transform(documents).toarray()

# Splitting the dataset into the Training set and Test set
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.4, random_state = 0)


In [26]:
#bigram_vectorizer.get_feature_names()

In [27]:
#print(X_Test)
#print(Y[1])

In [10]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=4)
classifier_LR.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện Logistic Regression: ", str(end-start), " giây")

Thời gian huấn luyện Logistic Regression:  662.2940666675568  giây


In [11]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes
import time
from sklearn.naive_bayes import BernoulliNB

start = time.time()

classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện Naive Bayes: ", str(end-start), " giây")

Thời gian huấn luyện Naive Bayes:  8.354334115982056  giây


In [12]:
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start = time.time()

classifier_RF = RandomForestClassifier(n_estimators=100, random_state=0)
classifier_RF.fit(X_Train, Y_Train) 

end = time.time()


print("Thời gian huấn luyện Random Forest: ", str(end-start), " giây")

Thời gian huấn luyện Random Forest:  3349.8310022354126  giây


In [13]:
# Xây dựng mô hình huấn luyện bằng KNN

from sklearn.neighbors import KNeighborsClassifier

start = time.time()

classifier_KNN = KNeighborsClassifier(n_neighbors=5)
classifier_KNN.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện KNN: ", str(end-start), " giây")

Thời gian huấn luyện KNN:  6067.075381994247  giây


In [ ]:
# Xây dựng mô hình huấn luyện bằng SVM
from sklearn.svm import SVC

start_SVM = time.time()

classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(X_Train, Y_Train)


end_SVM = time.time()
time_SVM = end_SVM-start_SVM

print("Thời gian huấn luyện: ", str(time_SVM), " giây")

In [14]:
# Predicting the test set results with LogisticRegresion
Y_Pred_LR = classifier_LR.predict(X_Test)

# Predicting the test set results with Naive Bayes
Y_Pred_NB = classifier_NB.predict(X_Test)

# Predicting the test set results with RF 
Y_Pred_RF = classifier_RF.predict(X_Test)

Y_Pred_KNN = classifier_KNN.predict(X_Test)
#Y_pred_SVM = classifier_SVM.predict(X_Test)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Accuracy LogisticRegression: ', accuracy_score(Y_Test, Y_Pred_LR))
print('Accuracy Naive Bayes: ', accuracy_score(Y_Test,Y_Pred_NB))
print('Accuracy RF: ', accuracy_score(Y_Test, Y_Pred_RF))
print('Accuracy KNN: ', accuracy_score(Y_Test, Y_Pred_KNN))
#print('Accuracy of SVM:',accuracy_score(Y_Test, Y_pred_SVM))

Accuracy LogisticRegression:  0.5480339214178508
Accuracy Naive Bayes:  0.5479467870283035
Accuracy RF:  0.5479358952296101
Accuracy KNN:  0.4931348992835375


In [33]:
# Cái này cũng giống accuracy_score
print(classifier_KNN.score(X_Test, Y_Test)) 

0.45454545454545453
